<a href="https://colab.research.google.com/github/jantic/DeOldify/blob/master/DeOldify_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeOldify on Colab #

This notebook allows you to colorize your own images using Google Colab!

Special thanks to the that made this possible!

Original Author:  Matt Robinson, <matthew67robinson@gmail.com>

Additional Contributions: Maria Benavente

In [ ]:
!git clone -b FastAIv1 --single-branch https://github.com/jantic/DeOldify.git DeOldify

In [ ]:
from os import path
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
cd DeOldify

In [ ]:
# Doing work so I can access data from my google drive
!pip install PyDrive

In [ ]:
!pip install tensorboardX

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
from IPython.display import Image
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.tensorboard import *
from fasterai.loss import *
from fasterai.filters import *
from fasterai.generators import *
from pathlib import Path
from itertools import repeat
from google.colab import drive
torch.backends.cudnn.benchmark=True

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Note that the above requires a verification step. It isn't too bad.

With access to your Google Drive, the "deOldifyImages" directory will be created. Drop there your personal images, and after the full execution of the notebook find the results at its subdirectory "results"

In [ ]:
weights_name = 'colorize_gen'
results_dir=Path('/content/drive/My Drive/deOldifyImages/results')

#Adjust this if image doesn't look quite right (max 64 on 11GB GPU).  The default here works for most photos.  
#It literally just is a number multiplied by 16 to get the square render resolution.  
#Note that this doesn't affect the resolution of the final output- the output is the same resolution as the input.
#Example:  render_factor=21 => color is rendered at 16x21 = 336x336 px.  
render_factor=21

In [ ]:
# Now download the pretrained weights, which I have saved to my google drive
# note that the id is the ending part of the shareable link url (after open?id=)
# The pretrained weights can be downloaded from https://www.dropbox.com/s/3e4dqky91h824ik/colorize_gen.pth
!mkdir "models"
download = drive.CreateFile({'id': '1ReNu8QCgi4vfyDovgYGMs7tlUGrnz72V'})
download.GetContentFile('./models/colorize_gen.pth')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir "/content/drive/My Drive/deOldifyImages"
!mkdir "/content/drive/My Drive/deOldifyImages/results"

In [ ]:
data = get_dummy_databunch()

In [ ]:
learn = colorize_gen_learner(data=data, nf_factor=1.25)
#switch to read models from proper place
learn.path = Path('./')
learn.load(weights_name)
learn.model.eval()
filtr = MasterFilter([ColorizerFilter(learn=learn)], render_factor=render_factor)
vis = ModelImageVisualizer(filtr, results_dir=results_dir)

Here's an example of colorizing an image downloaded from the internet:

In [ ]:
!wget "https://media.githubusercontent.com/media/jantic/DeOldify/master/test_images/TV1930s.jpg" -O "family_TV.jpg"

In [ ]:
vis.plot_transformed_image('family_TV.jpg')

If you want to colorize pictures from your Google drive, drop them in a directory named deOldifyImages (in the root of your drive) and the next cell will colorize all of them and save the resulting images in deOldifyImages/results.

In [ ]:
for img in os.listdir("/content/drive/My Drive/deOldifyImages/"):
  img_path = str("/content/drive/My Drive/deOldifyImages/") + img
  if os.path.isfile(img_path):
    vis.plot_transformed_image(img_path)